In [56]:
import pandas as pd

# read the entire file into a python array
with open('yelp_train_academic_dataset_business.json', 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
data_df = pd.read_json(data_json_str)

In [57]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [58]:
df_flat = []
for i in range(0,len(data_df)):
    df_flat.append(flatten(data_df['attributes'][i]))

In [9]:
df_flat[:5]

[{u'By Appointment Only': True},
 {u'Accepts Credit Cards': True,
  u'Alcohol': u'none',
  u'Ambience_casual': False,
  u'Ambience_classy': False,
  u'Ambience_divey': False,
  u'Ambience_hipster': False,
  u'Ambience_intimate': False,
  u'Ambience_romantic': False,
  u'Ambience_touristy': False,
  u'Ambience_trendy': False,
  u'Ambience_upscale': False,
  u'Attire': u'casual',
  u'Caters': False,
  u'Delivery': False,
  u'Good For Groups': True,
  u'Good For_breakfast': False,
  u'Good For_brunch': False,
  u'Good For_dessert': False,
  u'Good For_dinner': False,
  u'Good For_latenight': False,
  u'Good For_lunch': True,
  u'Good for Kids': True,
  u'Has TV': True,
  u'Noise Level': u'average',
  u'Outdoor Seating': False,
  u'Parking_garage': False,
  u'Parking_lot': True,
  u'Parking_street': False,
  u'Parking_valet': False,
  u'Parking_validated': False,
  u'Price Range': 1,
  u'Take-out': True,
  u'Takes Reservations': False,
  u'Waiter Service': True},
 {u'Accepts Credit Cards':

In [59]:
counter = 0
for i in df_flat:
    counter = counter +1
#    assert counter <500
    for key,value in i.iteritems():
        change_list=[]
        if value == True:
            i[key] = 1
        elif value == False:
            i[key] = 0
        else:
            change_list.append([key,value])
        for j in change_list:
          #  print j
            i[str(j[0]+'_'+str(j[1]))] = 1
            del i[j[0]]
       
  #  print i
df_flat[:5]

[{u'By Appointment Only': 1},
 {u'Accepts Credit Cards': 1,
  'Alcohol_none': 1,
  u'Ambience_casual': 0,
  u'Ambience_classy': 0,
  u'Ambience_divey': 0,
  u'Ambience_hipster': 0,
  u'Ambience_intimate': 0,
  u'Ambience_romantic': 0,
  u'Ambience_touristy': 0,
  u'Ambience_trendy': 0,
  u'Ambience_upscale': 0,
  'Attire_casual': 1,
  u'Caters': 0,
  u'Delivery': 0,
  u'Good For Groups': 1,
  u'Good For_breakfast': 0,
  u'Good For_brunch': 0,
  u'Good For_dessert': 0,
  u'Good For_dinner': 0,
  u'Good For_latenight': 0,
  u'Good For_lunch': 1,
  u'Good for Kids': 1,
  u'Has TV': 1,
  'Noise Level_average': 1,
  u'Outdoor Seating': 0,
  u'Parking_garage': 0,
  u'Parking_lot': 1,
  u'Parking_street': 0,
  u'Parking_valet': 0,
  u'Parking_validated': 0,
  u'Price Range': 1,
  u'Take-out': 1,
  u'Takes Reservations': 0,
  u'Waiter Service': 1},
 {u'Accepts Credit Cards': 1,
  u'Ambience_casual': 1,
  u'Ambience_classy': 0,
  u'Ambience_divey': 0,
  u'Ambience_hipster': 0,
  u'Ambience_inti

In [60]:
len(df_flat)

37938

In [61]:
len(data_df['stars'])

37938

In [194]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn import pipeline
from sklearn.linear_model import LinearRegression

vec = DictVectorizer()
vec.fit_transform(df_flat)
linreg = LinearRegression(fit_intercept=True) 
att_pipe = pipeline.Pipeline([
  ('dictV', vec),
  ('linreg', linreg)
  ])
att_pipe.fit(df_flat,data_df['stars'])



Pipeline(steps=[('dictV', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('linreg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [39]:
import dill
dill.dump(att_pipe, open("attr_model.pkl", "w"))

In [195]:
import sklearn as sk
class flat_Transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):        

    def fit(self, X, y=None):
        return self

    def transform(self, data_df):
        import collections
        def flatten(d, parent_key='', sep='_'):
            items = []
            for k, v in d.items():
                new_key = parent_key + sep + k if parent_key else k
                if isinstance(v, collections.MutableMapping):
                    items.extend(flatten(v, new_key, sep=sep).items())
                else:
                    items.append((new_key, v))
            return dict(items)
        df_flat = []
        for i in range(0,len(data_df)):
            df_flat.append(flatten(data_df['attributes'][i]))
    #    assert len(df_flat) == 37938
        for i in df_flat:
            for key,value in i.iteritems():
                change_list=[]
                if value == True:
                    i[key] = 1
                elif value == False:
                    i[key] = 0
                else:
                    change_list.append([key,value])
                for j in change_list:
                    i[str(j[0]+'_'+str(j[1]))] = 1
                    del i[j[0]]  
        return df_flat

In [197]:
flat_trans = flat_Transformer()
vec = DictVectorizer()
att_X = pipeline.Pipeline([
        ('flat',flat_trans),
        ('dictV',vec)
    ])

In [169]:
df_flat_2=flat_trans.transform(data_df)

In [198]:
att_pipe = pipeline.Pipeline([
  ('att_X_',att_X),
  ('linreg', linreg)
  ])
att_pipe.fit(data_df,data_df['stars'])

Pipeline(steps=[('att_X_', Pipeline(steps=[('flat', flat_Transformer()), ('dictV', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('linreg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [188]:
len(ans)

37938

In [171]:
import dill
dill.dump(att_pipe, open("attr_model.pkl", "w"))

In [199]:
record =     {"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", 
              "full_address": "4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018", 
              "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, 
              "open": True, 
              "categories": ["Doctors", "Health & Medical"], 
              "city": "Phoenix", "review_count": 7, 
              "name": "Eric Goldberg, MD", 
              "neighborhoods": [], 
              "longitude": -111.98375799999999, 
              "state": "AZ", "stars": 3.5, 
              "latitude": 33.499313000000001, 
              "attributes": {"By Appointment Only": True}, "type": "business"}
att_pipe.predict(pd.DataFrame([record]))[0]

4.037490120007126

In [204]:
##########################################categories transformer#################################
class CategoryTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, data_df):
        from sklearn.feature_extraction.text import TfidfVectorizer
        str1=[]
        for i in data_df['categories']:
            str1.append( str(' '.join(i)))
        tf = TfidfVectorizer(analyzer='word', stop_words = 'english')        
        return tf.fit_transform(str1).toarray()


In [205]:
#################lat long tranformer##############################
class ColumnSelectTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):

    def __init__(self):
        self.k = 0

    def fit(self, X):
        return self

    def transform(self, X):
        return np.array(X.loc[:,['latitude','longitude']]).astype(np.float)

In [207]:
cat = CategoryTransformer()
cat.transform(data_df).shape

(37938, 832)

In [158]:
#################city tranformer##############################
class CityTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, data_df):       
        return np.array(data_df['city'])

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', stop_words = 'english')  
tf.fit_transform(data_df.loc[:,['city']]).toarray()

array([[ 1.]])

In [161]:
import numpy as np
np.array(data_df['city'])

array([u'Phoenix', u'De Forest', u'De Forest', ..., u'Phoenix', u'Phoenix',
       u'Edinburgh'], dtype=object)

In [200]:
ct = CityTransformer()
ll = ColumnSelectTransformer()
cat = CategoryTransformer()
att_X = att_X


In [208]:
all_features = pipeline.FeatureUnion([
  ('ct', ct),
  ('ll', ll),
  ('cat',cat),
  ('att_X',att_X)
  ])
all_features.fit(data_df)
k_union = pipeline.Pipeline([
  ("features", all_features),
  ("linreg", LinearRegression(fit_intercept=True))
  ])
k_union.fit(data_df, data_df['stars'])
K_union.predict(pd.DataFrame([record]))

TypeError: fit() takes exactly 2 arguments (3 given)